### 마켓 데이터 크롤링
# KRX 종목 코드 전체 - 시가총액 순위

<img src="http://i.imgur.com/grQHNKG.jpg">
    
#### 2016~2018 FinanceData http://fb.com/financedata

# 시가총액 순위 정보
한국거래소(http://marketdata.krx.co.kr) 순위 정보에 전체 종목이 있으므로 종목 전체를 얻는데 사용할 수 도 있다.

* 시장정보 → 주식 → 순위정보 → 시가총액 상/하위
* http://marketdata.krx.co.kr/contents/MKD/04/0404/04040200/MKD04040200.jsp


<img width="60%" src="http://i.imgur.com/4QNFbly.png" >

# 시가총액 상/하위
* Excel (혹은 CSV)를 클릭하여 다운로드
* data.xls (혹은 data.csv) 파일 읽기

<img width="80%" src="http://i.imgur.com/1FTF3uG.png" >    

# 시가총액 데이터 다운로드 자동화

시가총액 데이터 파일 (data.xls)를 다운로드

1. http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx
1. http://file.krx.co.kr/download.jspx

## http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx

<img src="http://i.imgur.com/JNi82aa.png" >    

## http://file.krx.co.kr/download.jspx

<img src="http://i.imgur.com/tBKfdMD.png" >    

# 트래픽 분석

## 요청응답 1/2

http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx

* name:fileDown
* filetype:csv
* url:MKD/04/0404/04040200/mkd04040200_01
* market_gubun:ALL    #시장구분: ALL=전체, STK=코스피, KSQ=코스피, KNX=코넥스
* indx_ind_cd:             
* sect_tp_cd:
* schdate:20160601
* pagePath:/contents/MKD/04/0404/04040200/MKD04040200.jsp
    
    
## 요청응답 2/2

http://file.krx.co.kr/download.jspx
    
* code: [GenerateOTP.jspx 에서 반환된 문자열]


※ 요청응답 2/2 헤더에 Referer를 명시하여야 한다 (2018년 10월 26일)

In [1]:
import requests

# STEP 01: Generate OTP
gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
gen_otp_data = {
    'name':'fileDown',
    'filetype':'xls',
    'url':'MKD/04/0404/04040200/mkd04040200_01',
    'market_gubun':'ALL', #시장구분: ALL=전체, STK=코스피, KSQ=코스피, KNX=코넥스
    'indx_ind_cd':'', 
    'sect_tp_cd':'',
    'schdate':'20180601',
    'pagePath':'/contents/MKD/04/0404/04040200/MKD04040200.jsp',
}
    
r = requests.post(gen_otp_url, gen_otp_data)
code = r.content

# STEP 02: download
down_url = 'http://file.krx.co.kr/download.jspx'
down_data = {
    'code': code,
}

headers = {'Referer': 'http://marketdata.krx.co.kr'}
r = requests.post(down_url, down_data, headers=headers)
with open("marcap_data.xls", 'wb') as f:
    f.write(r.content)

In [2]:
import pandas as pd

pd.read_excel('marcap_data.xls').head()

,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가,고가,저가,시가총액,시가총액비중(%),상장주식수,외국인 보유주식수,외국인 지분율(%)
0,005930,삼성전자,"51,300",600,1.2,"13,038,499","665,063,170,392","50,500","51,700","49,950","329,311,357,110,000",17.10,"6,419,324,700","3,405,723,291",53.05
1,000660,SK하이닉스,"91,400","-2,000",-2.1,"4,790,644","435,439,684,800","90,800","92,200","89,600","66,539,416,161,000",3.46,"728,002,365","368,274,204",50.59
2,005935,삼성전자우,"40,700",200,0.5,"976,217","39,713,003,950","40,600","40,900","40,250","36,777,700,300,000",1.91,"903,629,000","760,518,760",84.16
3,068270,셀트리온,"273,500","8,000",3.0,"859,854","234,067,580,500","265,500","276,000","265,000","34,295,779,470,500",1.78,"125,395,903","27,266,390",21.74
4,005380,현대차,"140,000","1,000",0.7,"395,619","55,124,875,500","138,500","140,000","137,500","30,838,707,060,000",1.60,"220,276,479","99,888,542",45.35


# 정리
* stock_master_krx()
* 시가총액순위 정보를 DataFrame으로 반환

In [3]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from datetime import datetime

def stock_master_krx(date=None):
    date = datetime.today() if date == None else date
    date = date.strftime('%Y%m%d') if type(date) == datetime else date 

    # STEP 01: Generate OTP
    gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
    gen_otp_data = {
        'name':'fileDown',
        'filetype':'xls',
        'url':'MKD/04/0404/04040200/mkd04040200_01',
        'market_gubun':'ALL', #시장구분: ALL=전체
        'indx_ind_cd':'',
        'sect_tp_cd':'',
        'schdate': date,
        'pagePath':'/contents/MKD/04/0404/04040200/MKD04040200.jsp',
    }
    
    r = requests.post(gen_otp_url, gen_otp_data)
    code = r.content
    
    # STEP 02: download
    down_url = 'http://file.krx.co.kr/download.jspx'
    down_data = {
        'code': code,
    }
    
    headers = {'Referer': 'http://marketdata.krx.co.kr'}
    r = requests.post(down_url, down_data, headers=headers)
    df = pd.read_excel(BytesIO(r.content), header=0, thousands=',')
    return df

# 사용예

In [4]:
df = stock_master_krx(datetime(2018, 6, 1))
df.head(10)

,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가,고가,저가,시가총액,시가총액비중(%),상장주식수,외국인 보유주식수,외국인 지분율(%)
0,005930,삼성전자,51300,600,1.2,13038499,665063170392,50500,51700,49950,329311357110000,17.10,6419324700,3405723291,53.05
1,000660,SK하이닉스,91400,-2000,-2.1,4790644,435439684800,90800,92200,89600,66539416161000,3.46,728002365,368274204,50.59
2,005935,삼성전자우,40700,200,0.5,976217,39713003950,40600,40900,40250,36777700300000,1.91,903629000,760518760,84.16
3,068270,셀트리온,273500,8000,3.0,859854,234067580500,265500,276000,265000,34295779470500,1.78,125395903,27266390,21.74
4,005380,현대차,140000,1000,0.7,395619,55124875500,138500,140000,137500,30838707060000,1.60,220276479,99888542,45.35
5,207940,삼성바이오로직스,451500,16500,3.8,480497,218996139500,440000,468500,437000,29873497500000,1.55,66165000,6447146,9.74
6,005490,POSCO,339500,0,0.0,150102,51280385420,340000,346500,339500,29599930482500,1.54,87186835,50439119,57.85
7,051910,LG화학,337000,-1500,-0.4,109544,36992326000,338000,340000,332500,23789619591000,1.24,70592343,27753858,39.32
8,028260,삼성물산,125000,-500,-0.4,441220,55311522500,125000,126500,124500,23711255375000,1.23,189690043,20912906,11.02
9,035420,NAVER,680000,11000,1.6,91461,62175026384,674000,686000,672000,22414621720000,1.16,32962679,19579552,59.40


In [5]:
print('종목수:', len(df))

종목수: 2312


----
#### 2016~2018 FinanceData http://fb.com/financedata http://financedata.github.com